In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
events_df = pd.read_csv('events.csv')
demand_df = pd.read_csv('demand.csv')
supply_df = pd.read_csv('supply.csv')

In [10]:
events_df.columns

Index(['city', 'year', 'month', 'content_eng', 'content_ko', 'type',
       'target_audience', 'duration', 'size', 'frequency', 'index', 'day',
       'collaboration', 'end_year', 'end_month', 'end_day'],
      dtype='object')

In [11]:
demand_df.columns

Index(['city', 'year', 'month', 'month_name', 'unit', 'patients',
       'bilateral thyroidectomy patients', 'bilateral thyroidectomy unit',
       'aortic valve replacement patients', 'aortic valve replacement unit',
       'graft replacement of aorta patients',
       'graft replacement of aorta unit', 'pda ligation patients',
       'pda ligation unit', 'exploratory laparotomy patients',
       'exploratory laparotomy unit', 'laparoscopic cholecystectomy patients',
       'laparoscopic cholecystectomy unit',
       'pylorus preserving pancreaticoduodenectomy patients',
       'pylorus preserving pancreaticoduodenectomy unit',
       'transurethral resection of prostate patients',
       'transurethral resection of prostate unit', 'prostatectomy patients',
       'prostatectomy unit', 'total abdominal hysterectomy patients',
       'total abdominal hysterectomy unit',
       'low cervical cesarean section patients',
       'low cervical cesarean section unit', 'total hip replacement 

In [12]:
supply_df.columns

Index(['city', 'year', 'month', 'composition_rate', 'total_unit', 'women_unit',
       'men_unit'],
      dtype='object')

In [13]:
events_cities = events_df['city'].unique()
demand_cities = demand_df['city'].unique()
supply_cities = supply_df['city'].unique()

print(set(events_cities) - set(demand_cities))
print(set(events_cities) - set(supply_cities))


{nan}
{nan}
